In [1]:
import pandas as pd
import numpy as np
from splinter.browser import Browser
import re

In [2]:
br = Browser()

In [3]:
url = "http://www.tripadvisor.com/Hotels-g31310-Phoenix_Arizona-Hotels.html"

In [4]:
br.visit(url)

In [5]:
hotels_div = br.find_by_xpath('//*[@id="ACCOM_OVERVIEW"]/div/div/div/div')
titles_div = br.find_by_xpath('//*[@class="listing_title"]')

In [6]:
hotels_div

In [7]:
hotel_names = []
links = []

for title in titles_div:
    print(title.text)
    hotel_names.append(title.text)
    print(title.find_by_xpath('a')['href'])
    links.append(title.find_by_xpath('a')['href'])
    print('*'*50)

Holiday Inn Hotel & Suites Phoenix Airport
http://www.tripadvisor.com/Hotel_Review-g31310-d1657449-Reviews-Holiday_Inn_Hotel_Suites_Phoenix_Airport-Phoenix_Arizona.html
**************************************************
Arizona Grand Resort & Spa
http://www.tripadvisor.com/Hotel_Review-g31310-d112178-Reviews-Arizona_Grand_Resort_Spa-Phoenix_Arizona.html
**************************************************
Pointe Hilton Squaw Peak Resort
http://www.tripadvisor.com/Hotel_Review-g31310-d73957-Reviews-Pointe_Hilton_Squaw_Peak_Resort-Phoenix_Arizona.html
**************************************************
Pointe Hilton Tapatio Cliffs Resort
http://www.tripadvisor.com/Hotel_Review-g31310-d277997-Reviews-Pointe_Hilton_Tapatio_Cliffs_Resort-Phoenix_Arizona.html
**************************************************
Arizona Biltmore, A Waldorf Astoria Resort
http://www.tripadvisor.com/Hotel_Review-g31310-d115484-Reviews-Arizona_Biltmore_A_Waldorf_Astoria_Resort-Phoenix_Arizona.html
*******************

In [8]:
links

[u'http://www.tripadvisor.com/Hotel_Review-g31310-d1657449-Reviews-Holiday_Inn_Hotel_Suites_Phoenix_Airport-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d112178-Reviews-Arizona_Grand_Resort_Spa-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d73957-Reviews-Pointe_Hilton_Squaw_Peak_Resort-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d277997-Reviews-Pointe_Hilton_Tapatio_Cliffs_Resort-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d115484-Reviews-Arizona_Biltmore_A_Waldorf_Astoria_Resort-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d73943-Reviews-Royal_Palms_Resort_and_Spa-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d223860-Reviews-JW_Marriott_Desert_Ridge_Resort_Spa_Phoenix-Phoenix_Arizona.html',
 u'http://www.tripadvisor.com/Hotel_Review-g31310-d286542-Reviews-Marriott_s_Canyon_Villas_at_Desert_Ridge-Phoenix_Arizona.html',
 

## Now to scrape the reviews from a specific hotel

In [11]:
br.visit(links[0])

In [12]:
full_reviews = br.find_by_xpath('//div[contains(@class, "reviewSelector")]')

In [13]:
full_reviews

In [14]:
fullrev = full_reviews[0]

In [15]:
fullrev.find_by_xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]').text

u'Mary G\nLafayette'

In [19]:
member_info = fullrev.find_by_xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]')

In [20]:
member_info.find_by_xpath('div/div[contains(@class, "username mo")]').text

u'Mary G'

In [22]:
review = fullrev.find_by_xpath('div/div[@class="col2of2"]/div[@class="innerBubble"]')

In [24]:
review.find_by_xpath('div/div[contains(@class, "quote")]').text.strip()[1:-1]

u'\u201ctraveling through\u201d'

In [26]:
review.find_by_xpath('div/div[contains(@class, "rating")]/span/img')['alt'].split(' ')[0]

u'5'

In [28]:
review.find_by_xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]')['title']

u'September 11, 2015'

In [29]:
review.find_by_xpath('div/div[contains(@class, "entry")]').text.strip().replace("\n", "")

u'We had previously stayed at this hotel and both times we were very pleased with the stay. The location is excellent for road warriors as it it very accessible. Plenty of eating or shopping options close by.'

In [30]:
fullrev['id']

u'review_309423510'

In [34]:
br.find_by_xpath('//a[contains(@class, "next")]')

In [35]:
len(br.find_by_xpath('//a[contains(@class, "next")]')) > 0

True

In [36]:
br.find_by_xpath('//a[contains(@class, "next")]')['href']

u'http://www.tripadvisor.com/Hotel_Review-g31310-d74927-Reviews-or10-La_Quinta_Inn_Suites_Phoenix_Chandler-Phoenix_Arizona.html#REVIEWS'

In [33]:
for fullrev in full_reviews:
    # user name:
    member_info = fullrev.find_by_xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]')
    usrnm = member_info.find_by_xpath('div/div[contains(@class, "username mo")]')
    review = fullrev.find_by_xpath('div/div[@class="col2of2"]/div[@class="innerBubble"]')[0]
    title = review.find_by_xpath('div/div[contains(@class, "quote")]').text.strip()[1:-1]
    rating = review.find_by_xpath('div/div[contains(@class, "rating")]/span/img')['alt'].split(' ')[0]
    date = review.find_by_xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]')['title']
    rev = review.find_by_xpath('div/div[contains(@class, "entry")]').text.strip().replace("\n", "")
    if len(usrnm) > 0:
        #location = member_info.xpath('div[1]')[0].text_content()
        print('Username: {}'.format(str(usrnm[0].text).strip()))
    else:
        print('Username: A Trip Advisor Member')
    
    location = member_info.find_by_xpath('div[contains(@class, "location")]')
    if len(location) > 0:
        print('Location: {}'.format(str(location[0].text).strip()))
    else:
        print('Location: ')
    
    print('full review_id: {}'.format(fullrev['id']))
    try:
        rev_id = re.search('review_(\d+)$', fullrev['id']).group(1)
    except AttributeError:
        rev_id = ''

    print('review_id: {}'.format(rev_id))
    print('Title: {}'.format(title))
    print('Rating: {}'.format(rating))
    print('Date: {}'.format(date))
    print('Review:')
    print(rev)
    print('*'*50)


Username: Mary G
Location: Lafayette
full review_id: review_309423510
review_id: 309423510
Title: traveling through
Rating: 5
Date: September 11, 2015
Review:
We had previously stayed at this hotel and both times we were very pleased with the stay. The location is excellent for road warriors as it it very accessible. Plenty of eating or shopping options close by.
**************************************************
Username: Leo B
Location: san diego
full review_id: review_309144874
review_id: 309144874
Title: VISIT
Rating: 3
Date: September 10, 2015
Review:
overall satisfied with stay. please consider my suggestion for breakfast [more fruit variety]. Would be nice to have an elliptical machine at the small gym in place of the other thread-mill. My reservation was for two twin bed room but was surprise to be provided with a room with a king bed. Staff quickly resolved the issue by providing a...More
**************************************************
Username: bucketoferrs
Location: Phoen

In [9]:
def return_results(url, page):
    br.visit(url)
    full_reviews = br.find_by_xpath('//div[contains(@class, "reviewSelector")]')

    page_usernames = []
    page_locations = []
    page_titles = []
    page_ratings = []
    page_dates = []
    page_reviews = []
    page_review_ids = []

    for fullrev in full_reviews:
        # user name:
        member_info = fullrev.find_by_xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]')
        usrnm = member_info.find_by_xpath('div/div[contains(@class, "username mo")]')
        review = fullrev.find_by_xpath('div/div[@class="col2of2"]/div[@class="innerBubble"]')[0]
        title = review.find_by_xpath('div/div[contains(@class, "quote")]').text.strip()[1:-1]
        rating = review.find_by_xpath('div/div[contains(@class, "rating")]/span/img')['alt'].split(' ')[0]
        date = review.find_by_xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]')['title']
        rev = review.find_by_xpath('div/div[contains(@class, "entry")]').text.strip().replace("\n", "")
#         if len(usrnm) > 0:
#             #location = member_info.xpath('div[1]')[0].text_content()
#             print('Username: {}'.format(str(usrnm[0].text).strip()))
#         else:
#             print('Username: A Trip Advisor Member')

        location = member_info.find_by_xpath('div[contains(@class, "location")]')
#         if len(location) > 0:
#             print('Location: {}'.format(str(location[0].text).strip()))
#         else:
#             print('Location: ')

#        print('full review_id: {}'.format(fullrev['id']))
        try:
            rev_id = re.search('review_(\d+)$', fullrev['id']).group(1)
        except AttributeError:
            rev_id = ''

#         print('review_id: {}'.format(rev_id))
#         print('Title: {}'.format(title))
#         print('Rating: {}'.format(rating))
#         print('Date: {}'.format(date))
#         print('Review:')
#        print(rev)
#        print('*'*50)

        page_titles.append(title)
        page_ratings.append(ratings)
        page_dates.append(date)
        page_reviews.append(rev)
        page_review_ids.append(rev_id)

    if len(br.find_by_xpath('//a[contains(@class, "next")]')) > 0:
        url = br.find_by_xpath('//a[contains(@class, "next")]')['href']
        more_reviews = True
        page += 1
#        print('url and page updated.')
    else:
        more_reviews = False
    
    ret_dict = {'usrnms': page_usernames,
                'locs': page_locations,
                'ttls': page_titles,
                'rtngs': page_ratings,
                'dts': page_dates,
                'rvws': page_reviews,
                'revids': page_review_ids,
               'url': url,
               'more_reviews': more_reviews,
               'page': page}
    return ret_dict


In [10]:
usernames = []
locations = []
titles = []
ratings = []
dates = []
reviews = []
review_ids = []

columns = [

bigdf = 

url = links[0]
more_reviews = True
page = 1
while more_reviews:
    #print('*'*50)
    print('*'*50)
    print('Now on page {}'.format(page))
    #print('*'*50)

    try:
        ret_dict = return_results(url, page)
        usernames.append(ret_dict['usrnms'])
        locations.append(ret_dict['locs'])
        titles.append(ret_dict['ttls'])
        ratings.append(ret_dict['rtngs'])
        dates.append(ret_dict['dts'])
        reviews.append(ret_dict['rvws'])
        review_ids.append(ret_dict['revids'])
        url = ret_dict['url']
        more_reviews = ret_dict['more_reviews']
        page = ret_dict['page']
    except:
        print('Timed out! Trying that page again...' )
    more_reviews = False


**************************************************
Now on page 1
Nice front desk management-forgive me-I do not have his name. He was manager on duty Sept 4th about 9pm! We were greeted as real guests rather than folks staying a night at leaving our car at the hotel for purposes of airport parking. Unlike like some hotels that have the little breakfast rooms, this is a full service Holiday Inn...More
Overall pleasant experience. Close proximity to airport makes for a cheap cab ride. The hotel is in need of some modern renovations but the place was clean. Lucas at the front desk made me chuckle. Good job guys!
From the very start my mom and I were welcomed and greeted with a smile. Gaby was up at the front and made the check in process satisfactory. The room was nice, and had everything in it from the start. Lucas and Bree were back at the front later on when I had to go get an extra room key...More
We were a group of 5 needing a park and fly option. Everything from checkin to check out